# Experiment 003: bbox3 Optimization

Run the bbox3 C++ optimizer on the best available submission to improve the score.

In [1]:
import pandas as pd
import numpy as np
import math
from numba import njit
import subprocess
import os
import time
import json

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c, s = math.cos(r), math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx, mxx = min(mnx, X), max(mxx, X)
            mny, mxy = min(mny, Y), max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def calculate_total_score(df):
    total_score = 0.0
    scores_by_n = {}
    for n in range(1, 201):
        mask = df['id'].str.startswith(f'{n:03d}_')
        group = df[mask]
        if len(group) != n:
            continue
        xs = group['x'].str[1:].astype(float).values
        ys = group['y'].str[1:].astype(float).values
        degs = group['deg'].str[1:].astype(float).values
        score = score_group(xs, ys, degs, TX, TY)
        scores_by_n[n] = score
        total_score += score
    return total_score, scores_by_n

print("Scoring functions defined")

Scoring functions defined


In [2]:
# Calculate initial score
os.chdir('/home/code/experiments/003_bbox3_optimization')
df_initial = pd.read_csv('submission.csv')
initial_score, initial_scores_by_n = calculate_total_score(df_initial)
print(f"Initial Score: {initial_score:.6f}")
print(f"Expected: 70.624424")
print(f"Rows: {len(df_initial)}")

Initial Score: 70.624424
Expected: 70.624424
Rows: 20100


In [3]:
# Check bbox3 executable
bbox3_path = '/home/code/bbox3'
print(f"bbox3 exists: {os.path.exists(bbox3_path)}")
print(f"bbox3 executable: {os.access(bbox3_path, os.X_OK)}")

bbox3 exists: True
bbox3 executable: True


In [4]:
# Run bbox3 optimization - Phase A: Quick runs to test
# Parameters: -n iterations, -r rounds

def run_bbox3(n_iter, rounds, timeout=300):
    """Run bbox3 optimizer and return the improvement"""
    start_time = time.time()
    
    # Read initial score
    df_before = pd.read_csv('submission.csv')
    score_before, _ = calculate_total_score(df_before)
    
    # Run bbox3
    cmd = f'{bbox3_path} -n {n_iter} -r {rounds}'
    print(f"Running: {cmd}")
    
    try:
        result = subprocess.run(
            cmd.split(),
            cwd='/home/code/experiments/003_bbox3_optimization',
            capture_output=True,
            text=True,
            timeout=timeout
        )
        print(f"stdout: {result.stdout}")
        if result.stderr:
            print(f"stderr: {result.stderr}")
    except subprocess.TimeoutExpired:
        print(f"Timeout after {timeout}s")
    
    # Read final score
    df_after = pd.read_csv('submission.csv')
    score_after, _ = calculate_total_score(df_after)
    
    elapsed = time.time() - start_time
    improvement = score_before - score_after
    
    print(f"Score: {score_before:.6f} -> {score_after:.6f} (improvement: {improvement:.6f})")
    print(f"Time: {elapsed:.1f}s")
    
    return score_after, improvement

print("bbox3 runner defined")

bbox3 runner defined


In [5]:
# Phase A: Quick test runs (2 min each)
print("=" * 60)
print("PHASE A: Quick test runs")
print("=" * 60)

best_score = initial_score
results = []

# Test with moderate parameters first
for n_iter in [1000, 2000]:
    for rounds in [30, 60]:
        score, improvement = run_bbox3(n_iter, rounds, timeout=120)
        results.append({'n': n_iter, 'r': rounds, 'score': score, 'improvement': improvement})
        if score < best_score:
            best_score = score
        print()

PHASE A: Quick test runs


Running: /home/code/bbox3 -n 1000 -r 30


stdout: Loading submission.csv...
Initial Score: 70.624424
Running SA optimization (iterations=1000, r=30)...
Final Score: 70.624424
Improvement: 0.000000
Time: 91.07s
Saved to submission.csv



Score: 70.624424 -> 70.624424 (improvement: 0.000000)
Time: 92.2s



Running: /home/code/bbox3 -n 1000 -r 60


...

stdout: Loading submission.csv...
Initial Score: 70.624424
Running SA optimization (iterations=2000, r=30)...
Final Score: 70.624424
Improvement: 0.000000
Time: 93.51s
Saved to submission.csv



Score: 70.624424 -> 70.624424 (improvement: 0.000000)
Time: 94.7s



Running: /home/code/bbox3 -n 2000 -r 60


stdout: Loading submission.csv...
Initial Score: 70.624424
Running SA optimization (iterations=2000, r=60)...
Final Score: 70.624424
Improvement: 0.000000
Time: 92.97s
Saved to submission.csv



Score: 70.624424 -> 70.624424 (improvement: 0.000000)
Time: 94.1s



In [6]:
# Phase B: Longer runs if Phase A showed improvement
print("=" * 60)
print("PHASE B: Longer optimization runs")
print("=" * 60)

# Run with more aggressive parameters
for n_iter in [3000, 5000]:
    for rounds in [60, 90]:
        score, improvement = run_bbox3(n_iter, rounds, timeout=300)
        results.append({'n': n_iter, 'r': rounds, 'score': score, 'improvement': improvement})
        if score < best_score:
            best_score = score
        print()

PHASE B: Longer optimization runs


Running: /home/code/bbox3 -n 3000 -r 60


stdout: Loading submission.csv...
Initial Score: 70.624424
Running SA optimization (iterations=3000, r=60)...
Final Score: 70.624424
Improvement: 0.000000
Time: 95.60s
Saved to submission.csv



Score: 70.624424 -> 70.624424 (improvement: 0.000000)
Time: 96.7s



Running: /home/code/bbox3 -n 3000 -r 90


...

stdout: Loading submission.csv...
Initial Score: 70.624424
Running SA optimization (iterations=5000, r=60)...
Final Score: 70.624424
Improvement: 0.000000
Time: 101.95s
Saved to submission.csv



Score: 70.624424 -> 70.624424 (improvement: 0.000000)
Time: 103.1s



Running: /home/code/bbox3 -n 5000 -r 90


stdout: Loading submission.csv...
Initial Score: 70.624424
Running SA optimization (iterations=5000, r=90)...
Final Score: 70.624424
Improvement: 0.000000
Time: 101.91s
Saved to submission.csv



Score: 70.624424 -> 70.624424 (improvement: 0.000000)
Time: 103.0s



In [7]:
# Skip Phase C - no improvement seen in Phase A or B
# The solution is at a local optimum that bbox3 cannot escape

print("=" * 60)
print("OPTIMIZATION SUMMARY")
print("=" * 60)
print(f"Initial score: {initial_score:.6f}")
print(f"Final score: {best_score:.6f}")
print(f"Total improvement: {initial_score - best_score:.6f}")
print(f"\nAll runs:")
for r in results:
    print(f"  n={r['n']:5d}, r={r['r']:3d}: score={r['score']:.6f}, improvement={r['improvement']:.6f}")

print("\n" + "=" * 60)
print("CONCLUSION: bbox3 cannot improve the pre-optimized solution.")
print("The solution is at a local optimum.")
print("Need to try fundamentally different approaches:")
print("  1. Tessellation for large N")
print("  2. Novel algorithms (genetic, NFP)")
print("  3. Focus on specific N values with room for improvement")
print("=" * 60)

OPTIMIZATION SUMMARY
Initial score: 70.624424
Final score: 70.624424
Total improvement: 0.000000

All runs:
  n= 1000, r= 30: score=70.624424, improvement=0.000000
  n= 1000, r= 60: score=70.624424, improvement=0.000000
  n= 2000, r= 30: score=70.624424, improvement=0.000000
  n= 2000, r= 60: score=70.624424, improvement=0.000000
  n= 3000, r= 60: score=70.624424, improvement=0.000000
  n= 3000, r= 90: score=70.624424, improvement=0.000000
  n= 5000, r= 60: score=70.624424, improvement=0.000000
  n= 5000, r= 90: score=70.624424, improvement=0.000000

CONCLUSION: bbox3 cannot improve the pre-optimized solution.
The solution is at a local optimum.
Need to try fundamentally different approaches:
  1. Tessellation for large N
  2. Novel algorithms (genetic, NFP)
  3. Focus on specific N values with room for improvement


In [ ]:
# Save metrics and copy submission
import shutil

# The submission is unchanged from initial
final_score = initial_score

os.makedirs('/home/submission', exist_ok=True)
shutil.copy('submission.csv', '/home/submission/submission.csv')
print(f"Copied to /home/submission/submission.csv")

# Save metrics
metrics = {
    'cv_score': final_score,
    'initial_score': initial_score,
    'improvement': 0.0,
    'optimization_runs': results,
    'conclusion': 'bbox3 cannot improve pre-optimized solution - at local optimum'
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics saved. Final CV Score: {final_score:.6f}")

In [ ]:
# Phase C: Very long run
print("=" * 60)
print("PHASE C: Extended optimization")
print("=" * 60)

# One long run with high parameters
score, improvement = run_bbox3(8000, 120, timeout=600)
results.append({'n': 8000, 'r': 120, 'score': score, 'improvement': improvement})
if score < best_score:
    best_score = score

In [ ]:
# Summary
print("\n" + "=" * 60)
print("OPTIMIZATION SUMMARY")
print("=" * 60)
print(f"Initial score: {initial_score:.6f}")
print(f"Final score: {best_score:.6f}")
print(f"Total improvement: {initial_score - best_score:.6f}")
print(f"\nAll runs:")
for r in results:
    print(f"  n={r['n']:5d}, r={r['r']:3d}: score={r['score']:.6f}, improvement={r['improvement']:.6f}")

In [ ]:
# Calculate final score and verify
df_final = pd.read_csv('submission.csv')
final_score, final_scores_by_n = calculate_total_score(df_final)
print(f"\nFinal verified score: {final_score:.6f}")

# Copy to submission folder
import shutil
os.makedirs('/home/submission', exist_ok=True)
shutil.copy('submission.csv', '/home/submission/submission.csv')
print("Copied to /home/submission/submission.csv")

In [ ]:
# Save metrics
metrics = {
    'cv_score': final_score,
    'initial_score': initial_score,
    'improvement': initial_score - final_score,
    'optimization_runs': results
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics saved. Final CV Score: {final_score:.6f}")